<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/keras_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Layer

In [ ]:
from tensorflow.keras.layers import Dense, Layer, BatchNormalization, Dropout, concatenate
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import tensorflow.keras as keras

from sklearn.model_selection import train_test_split

import numpy as np

In [ ]:
dense = Dense(10)

In [ ]:
bn = BatchNormalization()

In [ ]:
isinstance(dense, Layer)

In [ ]:
# importante! settando questo attributo a False i suoi pesi non vengono aggiornati col gradient descent
dense.trainable

In [ ]:
dense.variables

In [ ]:
dense.name

In [ ]:
dense.input

In [ ]:
dense.output

In [ ]:
# inizializzazione dei pesi tramite build

In [ ]:
dense = Dense(10)

In [ ]:
dense.build((100,))

In [ ]:
dense.trainable_variables

In [ ]:
# settare l'inizializzazione

In [ ]:
dense = Dense(10, kernel_initializer='glorot_normal')

In [ ]:
dense = Dense(10, kernel_initializer=keras.initializers.GlorotNormal(seed=1))

In [ ]:
# inizializzazione mediante call

In [ ]:
dense = Dense(10)

In [ ]:
dense(np.random.rand(10, 45))

In [ ]:
bn(np.random.rand(10, 45))

In [ ]:
bn.weights

In [ ]:
bn.trainable_weights

In [ ]:
bn.non_trainable_weights

# Modelli

## Sequenziale

In [ ]:
# Posso specificare un input con il parametro input_shape del primo layer
# oppure includendo all'inizio un oggetto di tipo Input

model = Sequential(
    [
     Input(10),
     Dense(10, activation='relu', name='dense_1'),
     Dropout(0.1, name='dropout_1'),
     Dense(10, activation='relu', name='dense_2'),
     Dropout(0.1, name='dropout_2'),
     Dense(1, name='output')
    ]
)

In [ ]:
isinstance(model, Model)

In [ ]:
isinstance(model, Layer)

In [ ]:
model.layers

In [ ]:
dense_1 = model.get_layer(name='dense_1')

In [ ]:
dense_1.output

In [ ]:
dense_1.input

In [ ]:
model.summary()

In [ ]:
plot_model(model=model, show_shapes=True)

## Funzionale



In [ ]:
input_1 = Input(shape=(10,))
input_2 = Input(shape=(20,))

# prima sottorete
x = Dense(100, activation='relu')(input_1)
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
# seconda sottorete
y = Dense(100, activation='relu')(input_2)
y = Dense(100, activation='relu')(y)
y = Dense(100, activation='relu')(y)

conc = concatenate([x, y])
out = Dense(1)(conc)

model = Model(inputs=[input_1, input_2], outputs=out)

In [ ]:
model.summary()

In [ ]:
plot_model(model)

## Subclassing

In [ ]:
class RegressorMLP(Model):
    def __init__(self, num_hidden_layers, activation, units_x_layer, dropout_rate):
        super().__init__(name='RegressorMLP')
        self.hidden_layers = []
        self.dropouts = []
        for i in range(num_hidden_layers):
            self.hidden_layers.append(Dense(units=units_x_layer, activation=activation, name=f'dense_{i}'))
            self.dropouts.append(Dropout(dropout_rate, name=f'dropout_{i}'))
        self.last_layer = Dense(1, name=f'dense_{num_hidden_layers}')

    def call(self, inputs, training=False):
        x = inputs
        for dense, dropout in zip(self.hidden_layers, self.dropouts):
            print(dense.name, dropout.name)
            x = dense(x)
            x = dropout(x) if training else x

        x = self.last_layer(x)
        
        return x

In [ ]:
model = RegressorMLP(num_hidden_layers=4, activation='relu', units_x_layer=50, dropout_rate=0.3)

In [ ]:
model(np.random.rand(32, 10).astype(np.float32), training=True)

# Addestramento

In [ ]:
# come si addestra un modello

In [ ]:
N = 100_000 # num of samples
x = np.random.rand(N, 10)
y = x @ np.array([1, 4, 2, 3, 5, 1, 6, 1, 1, 2]).reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
model = Sequential(
    [
     Input(10),
     Dense(10, activation='relu', name='dense_1'),
     Dropout(0.1, name='dropout_1'),
     Dense(10, activation='relu', name='dense_2'),
     Dropout(0.1, name='dropout_2'),
     Dense(1, name='output')
    ]
)

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

In [ ]:
model.history.history

In [ ]:
model.metrics_names

In [ ]:
loss, mae = model.evaluate(x_test, y_test)
print(f'loss={loss}, mae={mae}')

In [ ]:
y_hat = model.predict(x_test[:100])
print(y_hat.shape)

# Salvataggio di un modello

In [ ]:
# salvataggio del modello (anche lo stato dell'optimizer)
model.save('checkpoint')

In [ ]:
model = keras.models.load_model('/content/checkpoint')

In [ ]:
# salvataggio dei pesi
model.save_weights('weights/w', save_format='tf')

In [ ]:
model.load_weights('weights/w', by_name=False)

# Callbacks

In [ ]:
model = RegressorMLP(num_hidden_layers=4, activation='relu', units_x_layer=50, dropout_rate=0.3)

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

In [ ]:
callbacks = [
    CSVLogger('logger.csv'),
    EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True),
    ModelCheckpoint('train/checkpoint{epoch:02d}-{val_loss:.2f}')
]

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2, callbacks=callbacks)